In [ ]:

!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import re

In [ ]:
!pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.6/237.6 KB 18.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import pandas as pd
import numpy as np
import json
import re
import sys
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

In [ ]:
playlistDF = pd.read_csv("/content/processed_data.csv")
playlistDF.drop(columns=["Unnamed: 0",'Unnamed: 0.1'], inplace = True)
playlistDF.head()

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,name,danceability,...,type,id,uri,track_href,analysis_url,duration_ms_y,time_signature,artist_pop,genres,track_pop
0,0,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863.0,The Cookbook,Throwbacks,0.904,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864.0,4.0,74.0,dance_pop hip_hop hip_pop pop pop_rap r&b rap ...,69.0
1,73,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863.0,The Cookbook,w o r k o u t,0.904,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864.0,4.0,74.0,dance_pop hip_hop hip_pop pop pop_rap r&b rap ...,69.0
2,14,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863.0,The Cookbook,party playlist,0.904,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864.0,4.0,74.0,dance_pop hip_hop hip_pop pop pop_rap r&b rap ...,69.0
3,42,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863.0,The Cookbook,Dance mix,0.904,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864.0,4.0,74.0,dance_pop hip_hop hip_pop pop pop_rap r&b rap ...,69.0
4,1,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863.0,The Cookbook,spin,0.904,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864.0,4.0,74.0,dance_pop hip_hop hip_pop pop pop_rap r&b rap ...,69.0


In [ ]:
playlistDF.dtypes

pos                   int64
artist_name          object
track_uri            object
artist_uri           object
track_name           object
album_uri            object
duration_ms_x       float64
album_name           object
name                 object
danceability        float64
energy              float64
key                 float64
loudness            float64
mode                float64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
type                 object
id                   object
uri                  object
track_href           object
analysis_url         object
duration_ms_y       float64
time_signature      float64
artist_pop          float64
genres               object
track_pop           float64
dtype: object

In [ ]:
# Drop song duplicates
def drop_duplicates(df):
    '''
    Drop duplicate songs
    '''
    df['artists_song'] = df.apply(lambda row: str(row['artist_name'])+str(row['track_name']),axis = 1)
    return df.drop_duplicates('artists_song')

songDF = drop_duplicates(playlistDF)
print("Are all songs unique: ",len(pd.unique(songDF.artists_song))==len(songDF))

Are all songs unique:  True


In [ ]:
# Select useful columns
def select_cols(df):
       '''
       Select useful columns
       '''
       return df[['artist_name','id','track_name','danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', "artist_pop", "genres", "track_pop"]]
songDF = select_cols(songDF)
songDF.head()

,artist_name,id,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,genres,track_pop
0,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),0.904,0.813,4.0,-7.105,0.0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,74.0,dance_pop hip_hop hip_pop pop pop_rap r&b rap ...,69.0
6,Britney Spears,6I9VzXrHxO9rA9A5euc8Ak,Toxic,0.774,0.838,5.0,-3.914,0.0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,84.0,dance_pop pop post-teen_pop,83.0
19,Beyoncé,0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,0.664,0.758,2.0,-6.583,0.0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,86.0,dance_pop pop r&b,25.0
46,Justin Timberlake,1AWQoqb9bSvzTjaLralEkT,Rock Your Body,0.892,0.714,4.0,-6.055,0.0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,82.0,dance_pop pop,79.0
55,Shaggy,1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,0.853,0.606,0.0,-4.596,1.0,0.0713,0.05610,0.000000,0.3130,0.654,94.759,75.0,pop_rap reggae_fusion,2.0


In [ ]:
def genre_preprocess(df):
    '''
    Preprocess the genre data
    '''
    df['genres_list'] = df['genres'].apply(lambda x: str(x).split(" "))
    return df
songDF = genre_preprocess(songDF)
songDF['genres_list'].head()

0     [dance_pop, hip_hop, hip_pop, pop, pop_rap, r&...
6                       [dance_pop, pop, post-teen_pop]
19                                [dance_pop, pop, r&b]
46                                     [dance_pop, pop]
55                             [pop_rap, reggae_fusion]
Name: genres_list, dtype: object

In [ ]:
def playlist_preprocess(df):
    '''
    Preprocess imported playlist
    '''
    df = drop_duplicates(df)
    df = select_cols(df)
    df = genre_preprocess(df)

    return df

In [ ]:
def getSubjectivity(text):
  '''
  Getting the Subjectivity using TextBlob
  '''
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  '''
  Getting the Polarity using TextBlob
  '''
  return TextBlob(text).sentiment.polarity

def getAnalysis(score, task="polarity"):
  '''
  Categorizing the Polarity & Subjectivity score
  '''
  if task == "subjectivity":
    if score < 1/3:
      return "low"
    elif score > 1/3:
      return "high"
    else:
      return "medium"
  else:
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

def sentiment_analysis(df, text_col):
  '''
  Perform sentiment analysis on text
  ---
  Input:
  df (pandas dataframe): Dataframe of interest
  text_col (str): column of interest
  '''
  df['subjectivity'] = df[text_col].apply(getSubjectivity).apply(lambda x: getAnalysis(x,"subjectivity"))
  df['polarity'] = df[text_col].apply(getPolarity).apply(getAnalysis)
  return df

In [ ]:
songDF['track_name'] = songDF['track_name'].astype(str)

In [ ]:
# Show result
sentiment = sentiment_analysis(songDF, "track_name")
sentiment.head()

,artist_name,id,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,genres,track_pop,genres_list,subjectivity,polarity
0,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),0.904,0.813,4.0,-7.105,0.0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,74.0,dance_pop hip_hop hip_pop pop pop_rap r&b rap ...,69.0,"[dance_pop, hip_hop, hip_pop, pop, pop_rap, r&...",low,Neutral
6,Britney Spears,6I9VzXrHxO9rA9A5euc8Ak,Toxic,0.774,0.838,5.0,-3.914,0.0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,84.0,dance_pop pop post-teen_pop,83.0,"[dance_pop, pop, post-teen_pop]",low,Neutral
19,Beyoncé,0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,0.664,0.758,2.0,-6.583,0.0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,86.0,dance_pop pop r&b,25.0,"[dance_pop, pop, r&b]",high,Negative
46,Justin Timberlake,1AWQoqb9bSvzTjaLralEkT,Rock Your Body,0.892,0.714,4.0,-6.055,0.0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,82.0,dance_pop pop,79.0,"[dance_pop, pop]",low,Neutral
55,Shaggy,1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,0.853,0.606,0.0,-4.596,1.0,0.0713,0.05610,0.000000,0.3130,0.654,94.759,75.0,pop_rap reggae_fusion,2.0,"[pop_rap, reggae_fusion]",low,Neutral


In [ ]:
def ohe_prep(df, column, new_name):
    '''
    Create One Hot Encoded features of a specific column
    ---
    Input:
    df (pandas dataframe): Spotify Dataframe
    column (str): Column to be processed
    new_name (str): new column name to be used

    Output:
    tf_df: One-hot encoded features
    '''

    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)
    return tf_df

In [ ]:
# One-hot encoding for the subjectivity
subject_ohe = ohe_prep(sentiment, 'subjectivity','subject')
subject_ohe.iloc[0]

subject|high      0
subject|low       1
subject|medium    0
Name: 0, dtype: uint8

In [ ]:
# TF-IDF implementation
tfidf = TfidfVectorizer()
tfidf_matrix =  tfidf.fit_transform(songDF['genres_list'].apply(lambda x: " ".join(x)))
genre_df = pd.DataFrame(tfidf_matrix.toarray())
genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]
genre_df.drop(columns='genre|unknown')
genre_df.reset_index(drop = True, inplace=True)
genre_df.iloc[0]

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


genre|_hip_hop            0.0
genre|a_cappella          0.0
genre|abstract_beats      0.0
genre|abstract_hip_hop    0.0
genre|acid_rock           0.0
                         ... 
genre|ye_ye               0.0
genre|york_indie          0.0
genre|zambian_hip_hop     0.0
genre|zhongguo_feng       0.0
genre|zolo                0.0
Name: 0, Length: 1320, dtype: float64

In [ ]:
print(songDF['artist_pop'].describe())


count    9718.000000
mean       66.376929
std        17.768949
min         0.000000
25%        58.000000
50%        69.000000
75%        79.000000
max       100.000000
Name: artist_pop, dtype: float64


In [ ]:
# Normalization
pop = songDF[["artist_pop"]].reset_index(drop = True)
scaler = MinMaxScaler()
pop_scaled = pd.DataFrame(scaler.fit_transform(pop), columns = pop.columns)
pop_scaled.head()

,artist_pop
0,0.74
1,0.84
2,0.86
3,0.82
4,0.75


In [ ]:
def create_feature_set(df, float_cols):
    '''
    Process spotify df to create a final set of features that will be used to generate recommendations
    ---
    Input:
    df (pandas dataframe): Spotify Dataframe
    float_cols (list(str)): List of float columns that will be scaled

    Output:
    final (pandas dataframe): Final set of features
    '''

    # Tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['genres_list'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]
    genre_df.drop(columns='genre|unknown') # drop unknown genre
    genre_df.reset_index(drop = True, inplace=True)

    # Sentiment analysis
    df = sentiment_analysis(df, "track_name")

    # One-hot Encoding
    subject_ohe = ohe_prep(df, 'subjectivity','subject') * 0.3
    polar_ohe = ohe_prep(df, 'polarity','polar') * 0.5
    key_ohe = ohe_prep(df, 'key','key') * 0.5
    mode_ohe = ohe_prep(df, 'mode','mode') * 0.5

    # Normalization
    # Scale popularity columns
    pop = df[["artist_pop","track_pop"]].reset_index(drop = True)
    scaler = MinMaxScaler()
    pop_scaled = pd.DataFrame(scaler.fit_transform(pop), columns = pop.columns) * 0.2

    # Scale audio columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    # Concanenate all features
    final = pd.concat([genre_df, floats_scaled, pop_scaled, subject_ohe, polar_ohe, key_ohe, mode_ohe], axis = 1)

    # Add song id
    final['id']=df['id'].values

    return final

In [ ]:
# Save the data and generate the features
float_cols = songDF.dtypes[songDF.dtypes == 'float64'].index.values
songDF.to_csv("allsong_data.csv", index = False)

# Generate features
complete_feature_set = create_feature_set(songDF, float_cols=float_cols)
complete_feature_set.to_csv("complete_feature.csv", index = False)
complete_feature_set.head()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,genre|_hip_hop,genre|a_cappella,genre|abstract_beats,genre|abstract_hip_hop,genre|acid_rock,genre|acoustic_blues,genre|acoustic_chill,genre|acoustic_cover,genre|acoustic_pop,genre|acoustic_rock,...,key|5.0,key|6.0,key|7.0,key|8.0,key|9.0,key|10.0,key|11.0,mode|0.0,mode|1.0,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0UaMYEvWZi0ZqiDOoHU3YI
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,6I9VzXrHxO9rA9A5euc8Ak
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0WqIKmW4BTrj3eJFmnCKMv
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,1AWQoqb9bSvzTjaLralEkT
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1lzr43nnXAijIGYnCT8M8H


In [ ]:
# Test playlist:  Mom's playlist
playlistDF_test = playlistDF[playlistDF['name']=="Mom's playlist"]
playlistDF_test.head()
playlistDF_test.to_csv("test_playlist.csv")

In [ ]:
def generate_playlist_feature(complete_feature_set, playlist_df):
    '''
    Summarize a user's playlist into a single vector
    ---
    Input:
    complete_feature_set (pandas dataframe): Dataframe which includes all of the features for the spotify songs
    playlist_df (pandas dataframe): playlist dataframe

    Output:
    complete_feature_set_playlist_final (pandas series): single vector feature that summarizes the playlist
    complete_feature_set_nonplaylist (pandas dataframe):
    '''

    # Find song features in the playlist
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]
    # Find all non-playlist song features
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(playlist_df['id'].values)]
    complete_feature_set_playlist_final = complete_feature_set_playlist.drop(columns = "id")
    return complete_feature_set_playlist_final.sum(axis = 0), complete_feature_set_nonplaylist

In [ ]:
# Generate the features
complete_feature_set_playlist_vector, complete_feature_set_nonplaylist = generate_playlist_feature(complete_feature_set, playlistDF_test)

In [ ]:
# Non-playlist features
complete_feature_set_nonplaylist[complete_feature_set_nonplaylist.isnull().any(axis=1)]


,genre|_hip_hop,genre|a_cappella,genre|abstract_beats,genre|abstract_hip_hop,genre|acid_rock,genre|acoustic_blues,genre|acoustic_chill,genre|acoustic_cover,genre|acoustic_pop,genre|acoustic_rock,...,key|5.0,key|6.0,key|7.0,key|8.0,key|9.0,key|10.0,key|11.0,mode|0.0,mode|1.0,id
9718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [ ]:
# Summarized playlist features
complete_feature_set_playlist_vector

genre|_hip_hop             0.0
genre|a_cappella           0.0
genre|abstract_beats       0.0
genre|abstract_hip_hop     0.0
genre|acid_rock            0.0
                          ... 
key|9.0                    3.5
key|10.0                   2.5
key|11.0                   3.0
mode|0.0                   9.0
mode|1.0                  30.0
Length: 1355, dtype: float64

In [ ]:
def generate_playlist_recos(df, features, nonplaylist_features):
    '''
    Generated recommendation based on songs in aspecific playlist.
    ---
    Input:
    df (pandas dataframe): spotify dataframe
    features (pandas series): summarized playlist feature (single vector)
    nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist

    Output:
    non_playlist_df_top_40: Top 40 recommendations for that playlist
    '''

    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)].dropna()
    # Find cosine similarity between the playlist and the complete song set
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).dropna().values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_20 = non_playlist_df.sort_values('sim',ascending = False).head(20)

    return non_playlist_df_top_20

In [ ]:
recommend = generate_playlist_recos(songDF, complete_feature_set_playlist_vector, complete_feature_set_nonplaylist)
recommend.head(10)

      genre|_hip_hop  genre|a_cappella  genre|abstract_beats  \
9718             0.0               0.0                   0.0   

      genre|abstract_hip_hop  genre|acid_rock  genre|acoustic_blues  \
9718                     0.0              0.0                   0.0   

      genre|acoustic_chill  genre|acoustic_cover  genre|acoustic_pop  \
9718                   0.0                   0.0                 0.0   

      genre|acoustic_rock  ...  key|4.0  key|5.0  key|6.0  key|7.0  key|8.0  \
9718                  0.0  ...      0.0      0.0      0.0      0.0      0.0   

      key|9.0  key|10.0  key|11.0  mode|0.0  mode|1.0  
9718      0.0       0.0       0.0       0.0       0.0  

[1 rows x 1355 columns]


,artist_name,id,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,artist_pop,genres,track_pop,genres_list,subjectivity,polarity,sim
28834,American Authors,64ybTt8CKxPdeXBNnu08Op,Believer,0.583,0.968,1.0,-2.909,1.0,0.0368,0.001410,...,0.1300,0.910,119.999,70.0,indie_poptimism modern_alternative_rock modern...,55.0,"[indie_poptimism, modern_alternative_rock, mod...",low,Neutral,0.784369
28926,Neon Trees,0K1KOCeJBj3lpDYxEX9qP2,Sleeping With A Friend,0.582,0.882,2.0,-4.256,1.0,0.0355,0.001890,...,0.3200,0.507,107.034,71.0,modern_alternative_rock modern_rock pop pop_ro...,59.0,"[modern_alternative_rock, modern_rock, pop, po...",low,Neutral,0.766315
14547,The 1975,5hc71nKsUgtwQ3z52KEKQk,Somebody Else,0.610,0.788,0.0,-5.724,1.0,0.0585,0.195000,...,0.1530,0.472,101.045,78.0,modern_alternative_rock modern_rock pop rock,75.0,"[modern_alternative_rock, modern_rock, pop, rock]",low,Neutral,0.751696
14555,The 1975,6OPOa3qlKoDUzGpS8MrcLi,This Must Be My Dream,0.592,0.840,9.0,-4.697,1.0,0.0364,0.019300,...,0.3270,0.679,103.026,78.0,modern_alternative_rock modern_rock pop rock,52.0,"[modern_alternative_rock, modern_rock, pop, rock]",low,Neutral,0.751663
19620,Andy Grammer,4Jz4bjXeiF2SXVj9P4YfY5,Keep Your Head Up,0.674,0.778,0.0,-5.367,1.0,0.0376,0.046900,...,0.1450,0.816,90.010,74.0,dance_pop modern_rock neo_mellow pop pop_rap p...,74.0,"[dance_pop, modern_rock, neo_mellow, pop, pop_...",low,Neutral,0.749893
28838,American Authors,7JXdE4b0hpNSlZmPyopgii,Hit It,0.645,0.919,7.0,-3.669,1.0,0.0853,0.002500,...,0.5270,0.770,120.057,70.0,indie_poptimism modern_alternative_rock modern...,0.0,"[indie_poptimism, modern_alternative_rock, mod...",low,Neutral,0.747811
17120,Twenty One Pilots,4RdRa3qq0GBSKjomxrkh5K,Guns For Hands,0.684,0.858,1.0,-4.266,1.0,0.0342,0.006940,...,0.0828,0.518,129.963,85.0,modern_rock rock,66.0,"[modern_rock, rock]",low,Neutral,0.744487
31834,The 1975,73jVPicY2G9YHmzgjk69ae,Robbers,0.621,0.692,4.0,-6.858,1.0,0.0291,0.000363,...,0.3350,0.381,99.806,78.0,modern_alternative_rock modern_rock pop rock,71.0,"[modern_alternative_rock, modern_rock, pop, rock]",low,Neutral,0.741489
7733,Vance Joy,7yq4Qj7cqayVTp3FF9CWbm,Riptide,0.484,0.731,1.0,-6.694,1.0,0.0379,0.431000,...,0.1510,0.510,101.654,78.0,folk-pop modern_rock pop pop_rock,83.0,"[folk-pop, modern_rock, pop, pop_rock]",low,Neutral,0.735976
3796,Young the Giant,7w5Ww1cW8U9v8Q3g4qLpVD,Repeat,0.656,0.806,1.0,-5.354,1.0,0.0433,0.045400,...,0.3410,0.572,149.975,69.0,modern_alternative_rock modern_rock pop_rock r...,45.0,"[modern_alternative_rock, modern_rock, pop_roc...",low,Neutral,0.735136
